In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Libraries

In [ ]:
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, roc_curve, auc

# Loading Dataset

In [ ]:
!ls ../input/ieee-fraud-detection/

In [ ]:
train_id = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")
train_tra = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")
test_id = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
test_tra = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")

data = train_tra.merge(train_id, how='left')
test = test_tra.merge(test_id, how='left')
# data = pd.concat([train_id,train_tra])
# test = pd.concat([test_id,test_tra])
# train_id.head()

In [ ]:
a=['id_17', 'id_02', 'id_19', 'id_05']
for i in a:
    data.drop(i, inplace = True, axis = 1)

In [ ]:
a=['isFraud', 'id_17', 'id_02', 'id_19', 'id_05']
for i in a:
    print(i in data)

In [ ]:
test.head()

In [ ]:
test_id = test['TransactionID']
test_id

In [ ]:
data.head()

In [ ]:
'isFraud' in data.columns

In [ ]:
test.head()

In [ ]:
del train_tra, train_id, test_tra, test_id

In [ ]:
data.shape

# Preprocessing

In [ ]:
data.isnull().sum()/len(data) * 100

In [ ]:
for i in data.columns:
    if data[i].isnull().sum() / len(data) * 100 > 80:
#         print(i)
        data.drop(i, inplace = True, axis = 1)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()/len(data) * 100

In [ ]:
for i in data.columns:
    data[i] = data[i].fillna(data[i].mode()[0])

In [ ]:
data.describe()

In [ ]:
data['isFraud'].value_counts()

# Seprating Related and Unrelated columns

In [ ]:
cor = data.corr()

In [ ]:
rel = cor['isFraud'].sort_values(ascending = False)
rel

In [ ]:
data.info()

In [ ]:
l = []
for i in range (len(rel)):
    if rel[i] > 0:
        l.append(rel.index[i])

Converting object into integers

In [ ]:
la = LabelEncoder()
o = []
for i in data.columns:
    if data.dtypes[i] == 'O':
        o.append(i)
o

In [ ]:
for i in o:
    data[i] = la.fit_transform(data[i])

In [ ]:
data.info()

# Building Model

In [ ]:
x = data[l]
x.drop('isFraud', inplace = True, axis = 1)
x.head()

In [ ]:
y = data['isFraud']
y

In [ ]:
del data

Class Balancing

In [ ]:
over = SMOTE()
x, y = over.fit_resample(x, y)

In [ ]:
y.value_counts()

In [ ]:
x.describe()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

# Applying Random Forest

In [ ]:
ra = RandomForestClassifier(n_estimators = 200)
ra.fit(x_train, y_train)

Checking accuracy on training data

In [ ]:
predicted = ra.predict(x_train)
print("Accuracy using Random Forest is {} %".format(accuracy_score(predicted, y_train)*100))

Checking accuracy on testing data

In [ ]:
predicted = ra.predict(x_test)
print("Accuracy using Random Forest is {} %".format(accuracy_score(predicted, y_test)*100))

Visualizing AUC curve

In [ ]:
fpr, tpr, threshold= roc_curve(y_test, predicted, pos_label=1)
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.show()
print("AUC value is {} ".format(auc(fpr, tpr)))

In [ ]:
x.shape

In [ ]:
del x_train, x_test, y_train, y_test

# Preprocessing of Testing Data

In [ ]:
test.shape

In [ ]:
for i in test.columns:
    if test[i].isnull().sum() / len(test) * 100 > 80:
#         print(i)
        test.drop(i, inplace = True, axis = 1)

In [ ]:
test.isnull().sum()/len(test) * 100

In [ ]:
for i in test.columns:
    test[i] = test[i].fillna(test[i].mode()[0])

In [ ]:
test.info()

In [ ]:
o = []
for i in test.columns:
    if test.dtypes[i] == 'O':
        o.append(i)
o

Converting objects into integers

In [ ]:
for i in o:
    test[i] = la.fit_transform(test[i])

In [ ]:
test.info()

In [ ]:
l.remove('isFraud')
l

# Making Predictions on Testing Data

In [ ]:
x_test_pre = test[l]
predicted = ra.predict(x_test_pre)

In [ ]:
x_test_pre.shape

In [ ]:
submit = pd.DataFrame(data = {'TransactionID' : test_id, 'isFraud' :predicted})
submit

Saving the final results

In [ ]:
submit.to_csv("submission1.csv", index = False)
submit.head()